In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import netCDF4
import geopandas as gpd
import numpy as np
import matplotlib
from mpl_toolkits.basemap import Basemap

The following function was taken from [here](http://stackoverflow.com/a/1060330)

In [1]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [2]:
import urllib

def getnetcdf(year, month, day):
    formated_date = year+month+day+".nc"
    url = 'http://water.weather.gov/precip/p_download_new/%s/%s/%s/nws_precip_conus_%s' % (year, month, day, formated_date)
    tmp = urllib.URLopener()
    tmp.retrieve(url, formated_date)

In [3]:
import os
os.chdir('./tmp')

In [4]:
start_date = date(2015, 10, 1)
end_date = date(2015, 10, 5)
for single_date in daterange(start_date, end_date):
    year =  single_date.strftime("%Y")
    month =  single_date.strftime("%m")
    day =  single_date.strftime("%d")
    formated =  str(year)+str(month)+str(day)+".nc"
    getnetcdf(year, month, day)
    nc = NetCDFFile(formated)

In [ ]:

testfile 
testfile.retrieve("http://water.weather.gov/precip/p_download_new/2016/02/02/nws_precip_conus_20160202.nc", "nws_precip_conus_20160202.nc")

In [ ]:
import urllib
testfile = urllib.URLopener()
testfile.retrieve("http://water.weather.gov/precip/p_download_new/2016/02/02/nws_precip_conus_20160202.nc", "nws_precip_conus_20160202.nc")

In [ ]:
url = 'http://water.weather.gov/precip/p_download_new/2016/02/02/nws_precip_1day_observed_shape_20160202.tar.gz'
filename = 'nws_precip_1day_observed_shape_20160202'
geodf = gpd.read_file(url, vfs='gzip://' + filename)

In [ ]:
def plot_conus():
    """ Create a plot of the Continential US. """
    m = Basemap(
        llcrnrlon=-130.0,
        llcrnrlat=20.0,
        urcrnrlon=-60.0,
        urcrnrlat=55.0,
        projection='mill',
        resolution='')
    m.drawcoastlines()
    m.drawcountries()
    m.drawstates()
    return m

# Colorbar with NSW Precip colors
nws_precip_colors = [
    "#04e9e7",  # 0.01 - 0.10 inches
    "#019ff4",  # 0.10 - 0.25 inches
    "#0300f4",  # 0.25 - 0.50 inches
    "#02fd02",  # 0.50 - 0.75 inches
    "#01c501",  # 0.75 - 1.00 inches
    "#008e00",  # 1.00 - 1.50 inches
    "#fdf802",  # 1.50 - 2.00 inches
    "#e5bc00",  # 2.00 - 2.50 inches
    "#fd9500",  # 2.50 - 3.00 inches
    "#fd0000",  # 3.00 - 4.00 inches
    "#d40000",  # 4.00 - 5.00 inches
    "#bc0000",  # 5.00 - 6.00 inches
    "#f800fd",  # 6.00 - 8.00 inches
    "#9854c6",  # 8.00 - 10.00 inches
    "#fdfdfd"   # 10.00+
]
precip_colormap = matplotlib.colors.ListedColormap(nws_precip_colors)


# latitude and longitudes of HRAP grid
HRAP_XOR = 14
HRAP_YOR = 10


def lat_lon_from_hrap(hrap_x, hrap_y):
    """ Calculate the latitude and longitude for a HRAP grid. """
    raddeg = 57.29577951
    earthrad = 6371.2
    stdlon = 105.
    mesh_len = 4.7625

    tlat = 60. / raddeg
    x = hrap_x - 401.
    y = hrap_y - 1601.
    rr = x * x + y * y
    gi = ((earthrad * (1 + np.sin(tlat))) / mesh_len)
    gi = gi * gi

    ll_y = np.arcsin((gi - rr) / (gi + rr)) * raddeg
    ang = np.arctan2(y, x) * raddeg
    if (ang < 0):
        ang = ang + 360.

    ll_x = 270 + stdlon - ang
    if (ll_x < 0):
        ll_x = ll_x + 360
    if (ll_x > 360):
        ll_x = ll_x - 360
    return ll_x, ll_y


lats = np.empty((813, 1051), dtype='float')
lons = np.empty((813, 1051), dtype='float')

for i in xrange(813):
    for j in xrange(1051):
        hrap_x = j + HRAP_XOR + 0.5
        hrap_y = i + HRAP_YOR + 0.5
        lon, lat = lat_lon_from_hrap(hrap_x, hrap_y)
        lats[i, j] = lat
        lons[i, j] = -lon
# read in the data, convert in inches
dset = netCDF4.Dataset('nws_precip_conus_20160202.nc')
precip = dset.variables['amountofprecip'][:]
precip_in = np.ma.masked_less(precip / 2540., 0.01)
# plot the data
m = plot_conus()
levels = [0.01, 0.1, 0.25, 0.50, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0,
          6.0, 8.0, 10., 20.0]
norm = matplotlib.colors.BoundaryNorm(levels, 15)
cax = m.pcolormesh(lons, lats, precip_in, latlon=True, norm=norm,
                   cmap=precip_colormap)
m.colorbar(cax)
plt.savefig('conus_plot.png', dpi=200)

In [ ]:
from mpl_toolkits.basemap import Basemap, cm
# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import matplotlib.pyplot as plt
# plot rainfall from NWS using special precipitation
# colormap used by the NWS, and included in basemap.

nc = NetCDFFile('../../../examples/nws_precip_conus_20061222.nc')
# data from http://water.weather.gov/precip/
prcpvar = nc.variables['amountofprecip']
data = 0.01*prcpvar[:]
latcorners = nc.variables['lat'][:]
loncorners = -nc.variables['lon'][:]
lon_0 = -nc.variables['true_lon'].getValue()
lat_0 = nc.variables['true_lat'].getValue()
# create figure and axes instances
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0.1,0.1,0.8,0.8])
# create polar stereographic Basemap instance.
m = Basemap(projection='stere',lon_0=lon_0,lat_0=90.,lat_ts=lat_0,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[2],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[2],\
            rsphere=6371200.,resolution='l',area_thresh=10000)
# draw coastlines, state and country boundaries, edge of map.
m.drawcoastlines()
m.drawstates()
m.drawcountries()
# draw parallels.
parallels = np.arange(0.,90,10.)
m.drawparallels(parallels,labels=[1,0,0,0],fontsize=10)
# draw meridians
meridians = np.arange(180.,360.,10.)
m.drawmeridians(meridians,labels=[0,0,0,1],fontsize=10)
ny = data.shape[0]; nx = data.shape[1]
lons, lats = m.makegrid(nx, ny) # get lat/lons of ny by nx evenly space grid.
x, y = m(lons, lats) # compute map proj coordinates.
# draw filled contours.
clevs = [0,1,2.5,5,7.5,10,15,20,30,40,50,70,100,150,200,250,300,400,500,600,750]
cs = m.contourf(x,y,data,clevs,cmap=cm.s3pcpn)
# add colorbar.
cbar = m.colorbar(cs,location='bottom',pad="5%")
cbar.set_label('mm')
# add title
plt.title(prcpvar.long_name+' for period ending '+prcpvar.dateofdata)
plt.show()